In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.models import Model,Sequential
from keras.layers import Input,Conv2D,MaxPooling2D,Dropout,Flatten,Dense,GlobalAveragePooling2D,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
traingen = Imgen(rescale=1./255,
                shear_range = 0.2,
                zoom_range = 0.2,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                fill_mode="nearest",
                validation_split=0.15)


testgen = Imgen(rescale=1./255)

In [ ]:
trainds = traingen.flow_from_directory("../input/gender-classification-dataset/Training",
                                      target_size=(96,96),
                                       seed=123,
                                       batch_size=32,
                                       subset="training"
                                      )
valds = traingen.flow_from_directory("../input/gender-classification-dataset/Validation",
                                      target_size=(96,96),
                                       seed=123,
                                       batch_size=32,
                                       subset="validation"
                                      )

In [ ]:
c = trainds.class_indices
classes = list(c.keys())
classes
image_input = Input(shape=(96,96,3))

x = Flatten()(image_input)

x = Dense(256,activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

image_output = Dense(2,activation="softmax")(x)

ann_model = Model(image_input,image_output)
ann_model.summary()

In [ ]:
ann_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
import tensorflow as tf

with tf.device('GPU:0'):
    hist = ann_model.fit(trainds,epochs=22,validation_data=valds)

In [ ]:
x = Conv2D(32,(3,3),activation = "relu")(image_input)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.2)(x)

x = Conv2D(64,(3,3),activation = "relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.2)(x)

x = Conv2D(128,(3,3),activation = "relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.2)(x)

x = Flatten()(x)

x = Dense(256,activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

image_output = Dense(2,activation="softmax")(x)

cnn_model = Model(image_input,image_output)

In [ ]:
cnn_model.summary()
cnn_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
import tensorflow as tf

with tf.device('GPU:0'):
    hist_cnn = cnn_model.fit(trainds,epochs=22,validation_data=valds)
plt.figure(figsize=(15,6))

plt.subplot(1,2,2)
plt.plot(hist_cnn.epoch,hist_cnn.history['accuracy'],label = 'Training')
plt.plot(hist_cnn.epoch,hist_cnn.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2, 1)
plt.plot(hist_cnn.epoch,hist_cnn.history['loss'],label = 'Training')
plt.plot(hist_cnn.epoch,hist_cnn.history['val_loss'],label = 'validation')


plt.title("Loss")
plt.legend()
plt.show()